# Data Cleaning and Preprocessing

## Introduction
This notebook focuses on cleaning and preprocessing the Airbnb datasets for Paris and Los Angeles. Our goal is to prepare the data for analysis by addressing missing values, removing duplicates, and ensuring consistency across different data files.

## Objectives
1. Load and inspect all datasets (Summary Listings, Detailed Listings, Detailed Reviews, Detailed Calendar, and Summary Reviews)
2. Handle missing values and outliers
3. Ensure data consistency (e.g., data types, date formats)
4. Merge relevant datasets for comprehensive analysis
5. Create clean, analysis-ready datasets for both Paris and Los Angeles

## Contents
1. Data Loading and Initial Inspection
2. Handling Missing Values
3. Outlier Detection and Treatment
4. Data Type Conversion and Consistency Checks
5. Merging Datasets
6. Final Data Validation and Export

Let's begin by importing the necessary libraries and loading our datasets.

In [2]:
import pandas as pd
import glob
import os
import numpy as np

### Merge archive data into one merged file

In [3]:
def merge_same_csv_files(directory_path):
    # Get all CSV files in the specified directory
    csv_files = glob.glob(os.path.join(directory_path, "*.csv"))
    
    # Check if there are any CSV files
    if not csv_files:
        print(f"No CSV files found in {directory_path}")
        return None
    
    # Read and store all dataframes in a list
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        dataframes.append(df)
        print(f"Read file: {file}")
    
    # Concatenate all dataframes
    merged_df = pd.concat(dataframes, ignore_index=True)
    
    # Remove duplicate rows based on 'id' column
    merged_df.drop_duplicates(subset='id', keep='first', inplace=True)
    
    return merged_df

# Specify the directory containing your CSV files
file_type = "listings_detailed"
directory_path = "raw_data/" + file_type

# Merge the CSV files
merged_data = merge_same_csv_files(directory_path)

if merged_data is not None:
    # Save the merged raw_data to a new CSV file
    output_file = "raw_data/merged/merged_"+file_type+".csv"
    merged_data.to_csv(output_file, index=False)
    print(f"Merged raw_data saved to {output_file}")
    print(f"Total rows in merged file: {len(merged_data)}")

Read file: raw_data/listings_detailed\10th_june_listing.csv
Read file: raw_data/listings_detailed\12_dec_2023_listing.csv
Read file: raw_data/listings_detailed\16_march_listing.csv
Read file: raw_data/listings_detailed\6th_sept_listing.csv
Merged raw_data saved to raw_data/merged/merged_listings_detailed.csv
Total rows in merged file: 118804


### Merge data files, such as listings and reviews

In [5]:
def load_data():
    listings = pd.read_csv('raw_data/merged/merged_listings_detailed.csv')
    reviews = pd.read_csv('raw_data/merged/merged_reviews.csv')
    print("Data loaded successfully.")
    print(f"Size of listings: {listings.size}")
    print(f"Size of reviews: {reviews.size}")
    return listings, reviews

def merge_data(listings, reviews):
    merged_data = pd.merge(listings, reviews, left_on='id', right_on='listing_id', how='left')
    merged_data.fillna({'reviews_per_month': 0}, inplace=True)
    print(f"Size of merged: {merged_data.size}")
    return merged_data

def save_data(merged_data):
    output_file = 'raw_data/merged/merged_lst_det+rev.csv'
    merged_data.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")

listings, reviews = load_data()
merged_data = merge_data(listings, reviews)
save_data(merged_data)

C:\Users\sjokk\AppData\Local\Temp\ipykernel_8504\2725490828.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv('raw_data/merged/merged_listings_detailed.csv')


Data loaded successfully.
Size of listings: 8910300
Size of reviews: 163376
Size of merged: 9147908
Merged data saved to raw_data/merged/merged_lst_det+rev.csv


### Clean data and set correct types

In [5]:
import pandas as pd
import numpy as np

def clean_airbnb_data(input_file, output_file, alldata=True):
    # Read the merged CSV file
    df = pd.read_csv(input_file)
    print(f"Original shape: {df.shape}")

    # Clean the data
    df = clean_merged_data(df, alldata)

    # Save the cleaned data
    df.to_csv(output_file, index=False)
    print(f"Cleaned data saved to {output_file}")

    # Print summary of changes
    print_summary(df)

def clean_merged_data(df, alldata):
    # Convert 'last_scraped' to datetime at the beginning
    if 'last_scraped' in df.columns:
        df['last_scraped'] = pd.to_datetime(df['last_scraped'], errors='coerce')
    else:
        print("Warning: 'last_scraped' column not found. Unable to remove duplicates based on most recent entry.")
        return df

    # Remove duplicates based on 'id', keeping the most recent entry
    original_count = len(df)
    df = df.sort_values('last_scraped', ascending=False).drop_duplicates(subset='id', keep='first')
    duplicate_count = original_count - len(df)
    print(f"Removed {duplicate_count} duplicate entries based on 'id'")

    # Function to clean price
    def clean_price(price):
        if pd.isna(price) or price == '':
            return np.nan
        return float(str(price).replace('$', '').replace(',', ''))

    # Function to clean license
    def clean_license(license):
        if pd.isna(license) or license == '':
            return 'Unknown'
        return str(license)

    # Clean the data
    df['price'] = df['price'].apply(clean_price)
    df['license'] = df['license'].apply(clean_license)
    
    # Convert remaining date columns to datetime
    date_columns = ['last_review', 'date']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Remove rows with critical missing data
    critical_columns = ['id', 'latitude', 'longitude', 'room_type']
    df = df.dropna(subset=critical_columns)

    # Convert id, host_id, and listing_id to int
    for col in ['id', 'host_id', 'listing_id']:
        if col in df.columns:
            df[col] = df[col].astype('Int64')  # Using Int64 to handle potential NaN values

    # Fill NaN values in reviews_per_month with 0
    if 'reviews_per_month' in df.columns:
        df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
    
    # Optional: remove unwanted columns if alldata is False
    if not alldata:
        df = remove_unwanted_columns(df)
    
    return df

def remove_unwanted_columns(df):
    cols_to_remove = ['listing_url', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'host_name', 'host_since', 'host_location','host_about', 'host_response_time','host_response_rate', 'host_acceptance_rate', 'host_neighbourhood', 'host_verifications','host_has_profile_pic', 'host_identity_verified', 'neighborhood_overview', ' neighbourhood', 'neighbourhood_group_cleansed', 'calendar_updated']
    
    columns_to_remove = [col for col in cols_to_remove if col in df.columns]
    df = df.drop(columns=columns_to_remove)
    print(f"Removed columns: {columns_to_remove}")
    return df

def print_summary(df):
    print("\nData Cleaning Summary:")
    print(f"Final shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    if 'price' in df.columns:
        print(f"Rows with missing prices: {df['price'].isna().sum()}")
    if 'license' in df.columns:
        print(f"Unique license values: {df['license'].nunique()}")
        print(f"Sample of unique license values: {df['license'].sample(min(5, df['license'].nunique())).tolist()}")
    if 'date' in df.columns:
        print(f"Date range of reviews: {df['date'].min()} to {df['date'].max()}")
    if 'id' in df.columns:
        print(f"Number of unique listings: {df['id'].nunique()}")
    if 'reviews_per_month' in df.columns:
        print(f"Average reviews per month: {df['reviews_per_month'].mean():.2f}")

# Usage
input_file = 'raw_data/merged/merged_listings_detailed.csv'
output_file = 'cleaned_data/cleaned_listings_data.csv'
clean_airbnb_data(input_file, output_file, alldata=False)

C:\Users\sjokk\AppData\Local\Temp\ipykernel_11108\1821433011.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Original shape: (118804, 75)
Removed 0 duplicate entries based on 'id'
Removed columns: ['listing_url', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_neighbourhood', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighborhood_overview', 'neighbourhood_group_cleansed', 'calendar_updated']
Cleaned data saved to cleaned_data/cleaned_listings_data.csv

Data Cleaning Summary:
Final shape: (118804, 56)
Columns: ['id', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'host_id', 'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'neighbourhood', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum

# Continued cleaning

In [9]:
input_file = 'cleaned_data/cleaned_listings_data.csv'
output_file = 'cleaned_data/prepared/processed_listings_data.csv'

# Read the merged CSV file
df = pd.read_csv(input_file)
print(f"Original shape: {df.shape}")

columns_to_remove = ['scrape_id', 'source', 'neighbourhood', 'description','license', 'bathrooms_text']
df = df.drop(columns=columns_to_remove)
print(f"Removed columns: {columns_to_remove}")

# Save the cleaned data
df.to_csv(output_file, index=False)
print(f"Cleaned data saved to {output_file}")

C:\Users\sjokk\AppData\Local\Temp\ipykernel_11108\613561500.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Original shape: (118804, 56)
Removed columns: ['scrape_id', 'source', 'neighbourhood', 'description', 'license', 'bathrooms_text']
Cleaned data saved to cleaned_data/prepared/processed_listings_data.csv


# Plotting missing data

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV data
df = pd.read_csv('cleaned_data/prepared/processed_listings_data.csv')
# Calculate the percentage of missing values for each column
missing_percentages = df.isnull().mean() * 100

# Sort the columns by percentage of missing values (descending order)
missing_percentages_sorted = missing_percentages.sort_values(ascending=False)

# Filter the dataframe to include only columns with missing values
columns_with_missing = missing_percentages[missing_percentages > 0].index
df_missing = df[columns_with_missing]

# Create a heatmap of missing values with a clearer color scheme (only for columns with missing values)
plt.figure(figsize=(12, 8))
sns.heatmap(df_missing.isnull(), cbar=False, yticklabels=False, cmap='Reds')
plt.title('Missing Values Heatmap (Columns with Missing Values Only)')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.savefig('missing_values_heatmap.png')
plt.close()

# Create a bar plot of missing value percentages (only for columns with missing values)
missing_percentages_filtered = missing_percentages_sorted[missing_percentages_sorted > 0]
plt.figure(figsize=(12, 8))
missing_percentages_filtered.plot(kind='bar')
plt.title('Percentage of Missing Values by Column')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('missing_values_percentage.png')
plt.close()

# Print summary of missing values
print("Columns with missing values:")
print(missing_percentages[missing_percentages > 0].sort_values(ascending=False))

print("\nTotal number of missing values:", df.isnull().sum().sum())
print("Total number of cells in the dataset:", df.size)
print("Percentage of missing values:", (df.isnull().sum().sum() / df.size) * 100)

# Identify columns with more than 50% missing values
columns_to_drop = missing_percentages[missing_percentages > 50].index.tolist()
print("\nColumns with more than 50% missing values:")
print(columns_to_drop)

# Save the list of columns to drop
with open('columns_to_drop.txt', 'w') as f:
    for column in columns_to_drop:
        f.write(f"{column}\n")

print("\nAnalysis complete. Check the generated images and 'columns_to_drop.txt' file.")

Columns with missing values:


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Handle ratings

In [5]:
import pandas as pd
import numpy as np

def prepare_ratings_data(df):
    """
    Prepare ratings data for analysis with explicit handling of unrated listings.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing Airbnb listing data
    
    Returns:
    pandas.DataFrame: DataFrame with additional rating metadata columns
    dict: Rating statistics and segments
    """
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    # Create rating status columns
    df_clean['has_ratings'] = df_clean['number_of_reviews'] > 0
    
    # Create review volume segments
    df_clean['review_volume'] = pd.cut(
        df_clean['number_of_reviews'],
        bins=[-np.inf, 0, 5, 20, np.inf],
        labels=['unrated', 'few_reviews', 'moderate_reviews', 'many_reviews']
    )
    
    # Calculate rating statistics only for rated listings
    rated_listings = df_clean[df_clean['has_ratings']]
    rating_cols = [col for col in df_clean.columns if col.startswith('review_scores_')]
    
    stats = {
        'segments': {
            'total_listings': len(df_clean),
            'rated_listings': len(rated_listings),
            'unrated_listings': len(df_clean) - len(rated_listings),
            'review_volume_distribution': df_clean['review_volume'].value_counts().to_dict()
        },
        'rating_stats': {
            col: {
                'mean': rated_listings[col].mean(),
                'median': rated_listings[col].median(),
                'std': rated_listings[col].std(),
                'q25': rated_listings[col].quantile(0.25),
                'q75': rated_listings[col].quantile(0.75)
            } for col in rating_cols
        }
    }
    
    return df_clean, stats

def get_rating_subset(df, min_reviews=None, include_unrated=False):
    """
    Get a subset of listings based on rating criteria.
    
    Parameters:
    df (pandas.DataFrame): Prepared DataFrame from prepare_ratings_data
    min_reviews (int, optional): Minimum number of reviews required
    include_unrated (bool): Whether to include unrated listings
    
    Returns:
    pandas.DataFrame: Filtered DataFrame
    """
    mask = pd.Series(True, index=df.index)
    
    if not include_unrated:
        mask &= df['has_ratings']
    
    if min_reviews is not None:
        mask &= df['number_of_reviews'] >= min_reviews
    
    return df[mask]

def print_rating_summary(stats):
    """Print a summary of the rating statistics."""
    print("=== Rating Data Summary ===")
    print(f"\nSegment Sizes:")
    print(f"Total listings: {stats['segments']['total_listings']:,}")
    print(f"Rated listings: {stats['segments']['rated_listings']:,}")
    print(f"Unrated listings: {stats['segments']['unrated_listings']:,}")
    
    print("\nReview Volume Distribution:")
    for segment, count in stats['segments']['review_volume_distribution'].items():
        print(f"  {segment}: {count:,}")
    
    print("\nRating Statistics (rated listings only):")
    for col, metrics in stats['rating_stats'].items():
        print(f"\n{col}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.2f}")
        
        
# Read your data
df = pd.read_csv('cleaned_data/prepared/processed_listings_data.csv')
# Prepare the data
df_prepared, stats = prepare_ratings_data(df)

print_rating_summary(stats)

# For visualizations of ratings, use only rated listings
rated_df = get_rating_subset(df_prepared, include_unrated=False)

# For analyses that need high-confidence ratings
high_confidence_df = get_rating_subset(df_prepared, min_reviews=5)

# For full market analysis, use all listings but handle ratings explicitly
all_df = get_rating_subset(df_prepared, include_unrated=True)

=== Rating Data Summary ===

Segment Sizes:
Total listings: 118,804
Rated listings: 76,235
Unrated listings: 42,569

Review Volume Distribution:
  unrated: 42,569
  few_reviews: 29,208
  many_reviews: 23,874
  moderate_reviews: 23,153

Rating Statistics (rated listings only):

review_scores_rating:
  mean: 4.69
  median: 4.83
  std: 0.46
  q25: 4.57
  q75: 5.00

review_scores_accuracy:
  mean: 4.74
  median: 4.88
  std: 0.43
  q25: 4.67
  q75: 5.00

review_scores_cleanliness:
  mean: 4.62
  median: 4.77
  std: 0.51
  q25: 4.50
  q75: 5.00

review_scores_checkin:
  mean: 4.79
  median: 4.92
  std: 0.42
  q25: 4.75
  q75: 5.00

review_scores_communication:
  mean: 4.81
  median: 4.96
  std: 0.40
  q25: 4.78
  q75: 5.00

review_scores_location:
  mean: 4.80
  median: 4.92
  std: 0.35
  q25: 4.75
  q75: 5.00

review_scores_value:
  mean: 4.60
  median: 4.71
  std: 0.49
  q25: 4.49
  q75: 4.92


# Handle accomodation features

In [24]:
import pandas as pd
import numpy as np

def impute_accommodation_features(df):
    """
    Impute missing bedroom and bed values using logical rules and accommodates as proxy.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing Airbnb listing data
    
    Returns:
    pandas.DataFrame: DataFrame with imputed values
    dict: Imputation statistics
    """
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    # Track imputation methods
    imputation_stats = {
        'bedrooms': {'single_room_rule': 0, 'accommodates_proxy': 0},
        'beds': {'single_room_rule': 0, 'accommodates_proxy': 0}
    }
    
    # Step 1: Apply single room rule
    single_room_types = ['Private room', 'Shared room']
    for feature in ['bedrooms', 'beds']:
        mask = (df_clean[feature].isna()) & (df_clean['room_type'].isin(single_room_types))
        df_clean.loc[mask, feature] = 1
        imputation_stats[feature]['single_room_rule'] = mask.sum()
    
    # Step 2: Use accommodates as proxy for remaining missing values
    for feature in ['bedrooms', 'beds']:
        # Calculate mean values by accommodates
        means_by_accommodates = df_clean.groupby('accommodates')[feature].mean()
        
        # Fill remaining missing values
        mask = df_clean[feature].isna()
        df_clean.loc[mask, feature] = df_clean.loc[mask, 'accommodates'].map(means_by_accommodates)
        imputation_stats[feature]['accommodates_proxy'] = mask.sum()
    
    # Step 3: Apply logical constraints
    # Ensure beds >= bedrooms
    df_clean['beds'] = df_clean.apply(
        lambda x: max(x['beds'], x['bedrooms']) if pd.notnull(x['bedrooms']) else x['beds'],
        axis=1
    )
    
    # Add imputation flags
    df_clean['bedrooms_imputed_method'] = 'original'
    mask = df_clean['bedrooms'] != df['bedrooms']
    df_clean.loc[mask & (df_clean['room_type'].isin(single_room_types)), 'bedrooms_imputed_method'] = 'single_room_rule'
    df_clean.loc[mask & (~df_clean['room_type'].isin(single_room_types)), 'bedrooms_imputed_method'] = 'accommodates_proxy'
    
    df_clean['beds_imputed_method'] = 'original'
    mask = df_clean['beds'] != df['beds']
    df_clean.loc[mask & (df_clean['room_type'].isin(single_room_types)), 'beds_imputed_method'] = 'single_room_rule'
    df_clean.loc[mask & (~df_clean['room_type'].isin(single_room_types)), 'beds_imputed_method'] = 'accommodates_proxy'
    
    return df_clean, imputation_stats

def print_imputation_summary(df_original, df_imputed, stats):
    """Print summary of the imputation process and results."""
    print("=== Imputation Summary ===\n")
    
    # Missing values before
    print("Missing Values Before Imputation:")
    print(f"Bedrooms: {df_original['bedrooms'].isna().sum():,}")
    print(f"Beds: {df_original['beds'].isna().sum():,}\n")
    
    # Imputation methods used
    print("Imputation Methods Used:")
    for feature, methods in stats.items():
        print(f"\n{feature.title()}:")
        for method, count in methods.items():
            print(f"  {method}: {count:,}")
    
    # Distribution comparison
    print("\nValue Distribution After Imputation:")
    for feature in ['bedrooms', 'beds']:
        print(f"\n{feature.title()}:")
        print(df_imputed[feature].value_counts().sort_index().head())
    
    # Validation checks
    print("\nValidation Checks:")
    print(f"Beds >= Bedrooms: {(df_imputed['beds'] >= df_imputed['bedrooms']).mean()*100:.2f}%")
    print(f"Single rooms with 1 bedroom: {(df_imputed[df_imputed['room_type'].isin(['Private room', 'Shared room'])]['bedrooms'] == 1).mean()*100:.2f}%")
    
df = pd.read_csv('cleaned_data/prepared/processed_listings_data.csv')
# Apply the imputation
df_cleaned, imputation_stats = impute_accommodation_features(df)

# Print the summary
print_imputation_summary(df, df_cleaned, imputation_stats)

# Save the cleaned data
df_cleaned.to_csv("cleaned_data/prepared/acc_processed_listings_data.csv", index=False)

=== Imputation Summary ===

Missing Values Before Imputation:
Bedrooms: 17,627
Beds: 22,810

Imputation Methods Used:

Bedrooms:
  single_room_rule: 4,059
  accommodates_proxy: 13,568

Beds:
  single_room_rule: 3,474
  accommodates_proxy: 19,336

Value Distribution After Imputation:

Bedrooms:
bedrooms
0.000000     8561
0.876278     7574
0.934118      427
1.000000    67026
1.120280     1152
Name: count, dtype: int64

Beds:
beds
0.000000    1532
0.876278      13
0.911110    3128
0.934118       2
0.972815     237
Name: count, dtype: int64

Validation Checks:
Beds >= Bedrooms: 100.00%
Single rooms with 1 bedroom: 90.54%


NameError: name 'output_file' is not defined

# Analyzing bathrooms

In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

def analyze_missing_bathrooms(df):
    """
    Analyze patterns of missing bathroom data across different room types
    
    Parameters:
    df: pandas DataFrame with Airbnb listing data
    
    Returns:
    dict: Analysis results
    """
    # Create a copy to avoid modifying original
    data = df.copy()
    
    # Basic missing data analysis by room type
    missing_by_room = data.groupby('room_type').agg({
        'bathrooms': lambda x: x.isna().sum(),
        'id': 'count'  # Total count using ID column
    }).rename(columns={
        'bathrooms': 'missing_bathrooms',
        'id': 'total_listings'
    })
    
    # Calculate percentages
    missing_by_room['missing_percentage'] = (
        missing_by_room['missing_bathrooms'] / missing_by_room['total_listings'] * 100
    ).round(2)
    
    # Property type and room type combination analysis
    missing_by_prop_room = data.groupby(['property_type', 'room_type']).agg({
        'bathrooms': lambda x: x.isna().sum(),
        'id': 'count'
    }).rename(columns={
        'bathrooms': 'missing_bathrooms',
        'id': 'total_listings'
    }).reset_index()
    
    missing_by_prop_room['missing_percentage'] = (
        missing_by_prop_room['missing_bathrooms'] / missing_by_prop_room['total_listings'] * 100
    ).round(2)
    
    # Get top 10 property types with highest missing percentages
    # Only consider combinations with at least 10 listings
    top_missing = (
        missing_by_prop_room[missing_by_prop_room['total_listings'] >= 10]
        .sort_values('missing_percentage', ascending=False)
        .head(10)
    )
    
    # Analyze if missing bathrooms correlate with other features
    correlation_data = data.copy()
    correlation_data['has_bathroom'] = correlation_data['bathrooms'].notna().astype(int)
    
    # Select relevant numeric columns for correlation
    numeric_cols = []
    for col in ['accommodates', 'bedrooms', 'beds', 'price', 
                'minimum_nights', 'number_of_reviews', 'has_bathroom']:
        if col in correlation_data.columns:
            # Convert to numeric if possible
            correlation_data[col] = pd.to_numeric(
                correlation_data[col].replace('[\$,]', '', regex=True),
                errors='coerce'
            )
            numeric_cols.append(col)
    
    correlation_matrix = correlation_data[numeric_cols].corr()['has_bathroom'].sort_values()
    
    results = {
        'room_type_summary': missing_by_room,
        'property_room_details': missing_by_prop_room,
        'top_missing_combinations': top_missing,
        'feature_correlations': correlation_matrix,
        'total_listings': len(data),
        'total_missing': data['bathrooms'].isna().sum(),
        'overall_missing_percentage': (data['bathrooms'].isna().sum() / len(data) * 100).round(2)
    }
    
    return results

def plot_missing_bathroom_analysis(results):
    """
    Create visualizations for missing bathroom analysis
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Missing bathrooms by room type
    room_stats = results['room_type_summary']
    bars = ax1.bar(
        range(len(room_stats.index)),
        room_stats['missing_percentage']
    )
    ax1.set_xticks(range(len(room_stats.index)))
    ax1.set_xticklabels(room_stats.index, rotation=45)
    ax1.set_title('Percentage of Missing Bathrooms by Room Type')
    ax1.set_ylabel('Missing Percentage')
    
    # Add percentage labels on bars
    for bar in bars:
        height = bar.get_height()
        ax1.text(
            bar.get_x() + bar.get_width()/2.,
            height,
            f'{height:.1f}%',
            ha='center',
            va='bottom'
        )
    
    # Plot 2: Top 10 property-room combinations with missing bathrooms
    top_missing = results['top_missing_combinations']
    labels = [f"{prop}\n({room})" for prop, room in 
             zip(top_missing['property_type'], top_missing['room_type'])]
    
    bars = ax2.bar(range(len(labels)), top_missing['missing_percentage'])
    ax2.set_xticks(range(len(labels)))
    ax2.set_xticklabels(labels, rotation=45, ha='right')
    ax2.set_title('Top Property-Room Combinations\nwith Missing Bathrooms')
    ax2.set_ylabel('Missing Percentage')
    
    # Add percentage labels
    for bar in bars:
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width()/2.,
            height,
            f'{height:.1f}%',
            ha='center',
            va='bottom'
        )
    
    plt.tight_layout()
    return fig

def save_room_type_plot(results, filename='missing_bathrooms_by_room_type.png', dpi=300):
    """
    Create and save a bar plot showing percentage of missing bathrooms by room type
    
    Parameters:
    results: dict containing analysis results
    filename: str, name of file to save (default: 'missing_bathrooms_by_room_type.png')
    dpi: int, resolution of saved image (default: 300)
    """
    # Create figure and axis
    plt.figure(figsize=(10, 6))
    
    # Get room type statistics
    room_stats = results['room_type_summary']
    
    # Create bars
    bars = plt.bar(
        range(len(room_stats.index)),
        room_stats['missing_percentage']
    )
    
    # Customize plot
    plt.xticks(range(len(room_stats.index)), room_stats.index, rotation=45)
    plt.title('Percentage of Missing Bathrooms by Room Type', pad=20)
    plt.ylabel('Missing Percentage')
    
    # Add percentage labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2.,
            height,
            f'{height:.1f}%',
            ha='center',
            va='bottom'
        )
    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi, bbox_inches='tight')
    plt.close()

# Usage example:
# results = analyze_missing_bathrooms(df)
# save_room_type_plot(results)

df = pd.read_csv('cleaned_data/prepared/acc_processed_listings_data.csv')
# Assuming your DataFrame is called 'airbnb_df'
results = analyze_missing_bathrooms(df)

# Print overall statistics
print(f"Total listings analyzed: {results['total_listings']}")
print(f"Total missing bathroom values: {results['total_missing']}")
print(f"Overall missing percentage: {results['overall_missing_percentage']}%\n")

print("Missing Bathrooms by Room Type:")
print(results['room_type_summary'])

print("\nTop 10 Property-Room Combinations with Highest Missing Percentages:")
print(results['top_missing_combinations'][['property_type', 'room_type', 'missing_percentage']])

# Create visualizations
save_room_type_plot(results, "visualizations/missing_bathrooms_by_room_type.png")

<>:65: SyntaxWarning: invalid escape sequence '\$'
<>:65: SyntaxWarning: invalid escape sequence '\$'
C:\Users\sjokk\AppData\Local\Temp\ipykernel_17888\2260720592.py:65: SyntaxWarning: invalid escape sequence '\$'
  correlation_data[col].replace('[\$,]', '', regex=True),


Total listings analyzed: 118804
Total missing bathroom values: 33788
Overall missing percentage: 28.44%

Missing Bathrooms by Room Type:
                 missing_bathrooms  total_listings  missing_percentage
room_type                                                             
Entire home/apt              28513          106003               26.90
Hotel room                     279             834               33.45
Private room                  4704           11375               41.35
Shared room                    292             592               49.32

Top 10 Property-Room Combinations with Highest Missing Percentages:
                  property_type        room_type  missing_percentage
46           Room in aparthotel     Private room               76.09
47    Room in bed and breakfast       Hotel room               71.43
65        Shared room in hostel      Shared room               60.96
69   Shared room in rental unit      Shared room               47.67
37  Private room in ren

# Fixing bathrooms

In [37]:
import pandas as pd
import numpy as np

def impute_bathrooms(df):
    """
    Impute bathroom values for hotel rooms and private rooms while preserving
    other room types to maintain data integrity.
    
    Parameters:
    df: pandas DataFrame with Airbnb listing data
    
    Returns:
    pandas DataFrame: Copy of input data with imputed bathroom values
    """
    # Create a copy to avoid modifying original
    data = df.copy()
    
    # Store original missing counts for reporting
    original_missing = data['bathrooms'].isna().sum()
    
    # Create mask for rooms we want to impute
    hotel_mask = (data['property_type'] == 'Hotel') & (data['bathrooms'].isna())
    private_room_mask = (data['room_type'] == 'Private room') & (data['bathrooms'].isna())
    
    # Impute values
    data.loc[hotel_mask, 'bathrooms'] = 1.0
    data.loc[private_room_mask, 'bathrooms'] = 1.0
    
    # Calculate imputation statistics
    imputed_count = hotel_mask.sum() + private_room_mask.sum()
    remaining_missing = data['bathrooms'].isna().sum()
    
    # Create summary statistics
    summary = {
        'original_missing': original_missing,
        'imputed_count': imputed_count,
        'remaining_missing': remaining_missing,
        'imputed_percentage': (imputed_count / original_missing * 100).round(2) if original_missing > 0 else 0,
        'hotel_rooms_imputed': hotel_mask.sum(),
        'private_rooms_imputed': private_room_mask.sum()
    }
    
    # Add imputation method column if you want to track how values were imputed
    data['bathrooms_imputed_method'] = np.where(hotel_mask, 'hotel_rule',
                                              np.where(private_room_mask, 'private_room_rule',
                                                      'original'))
    
    return data, summary


df = pd.read_csv('cleaned_data/prepared/acc_processed_listings_data.csv')
df_imputed, imputation_summary = impute_bathrooms(df)

print(f"Original missing values: {imputation_summary['original_missing']}")
print(f"Values imputed: {imputation_summary['imputed_count']}")
print(f"Remaining missing values: {imputation_summary['remaining_missing']}")
print(f"Percentage of missing values imputed: {imputation_summary['imputed_percentage']}%")
print(f"Hotel rooms imputed: {imputation_summary['hotel_rooms_imputed']}")
print(f"Private rooms imputed: {imputation_summary['private_rooms_imputed']}")

df_cleaned.to_csv("cleaned_data/prepared/acc_bath_processed_listings_data.csv", index=False)

Original missing values: 33788
Values imputed: 4704
Remaining missing values: 29084
Percentage of missing values imputed: 13.92%
Hotel rooms imputed: 0
Private rooms imputed: 4704


# Fix empty host entries

In [39]:
import pandas as pd
import numpy as np

def clean_host_and_availability_data(df):
    """
    Clean and fix missing or problematic values in host and availability related columns
    
    Parameters:
    df: pandas DataFrame with Airbnb listing data
    
    Returns:
    pandas DataFrame: Copy of input data with cleaned values
    dict: Summary of changes made
    """
    # Create a copy to avoid modifying original
    data = df.copy()
    
    # Store original missing counts
    original_missing = {
        'host_is_superhost': data['host_is_superhost'].isna().sum(),
        'has_availability': data['has_availability'].isna().sum(),
        'host_total_listings_count': data['host_total_listings_count'].isna().sum(),
        'host_listings_count': data['host_listings_count'].isna().sum()
    }
    
    # 1. Fix host_is_superhost
    # Convert to boolean representation (f -> False, t -> True, empty -> False)
    data['host_is_superhost'] = data['host_is_superhost'].fillna('f')
    data['host_is_superhost'] = (data['host_is_superhost'] == 't')
    
    # 2. Fix has_availability
    # If availability_30 > 0, then has_availability should be true
    data['has_availability'] = data['has_availability'].fillna('')
    data.loc[data['availability_30'] > 0, 'has_availability'] = 't'
    data.loc[data['availability_30'] == 0, 'has_availability'] = 'f'
    
    # 3. Fix host listing counts
    # If one is missing but the other isn't, use the non-missing value
    data['host_total_listings_count'] = data['host_total_listings_count'].fillna(data['host_listings_count'])
    data['host_listings_count'] = data['host_listings_count'].fillna(data['host_total_listings_count'])
    
    # If both are missing, use calculated values if available
    calc_total = (data['calculated_host_listings_count_entire_homes'] + 
                 data['calculated_host_listings_count_private_rooms'] + 
                 data['calculated_host_listings_count_shared_rooms'])
    
    mask_missing_both = (data['host_total_listings_count'].isna() & 
                        data['host_listings_count'].isna())
    
    data.loc[mask_missing_both, 'host_total_listings_count'] = calc_total[mask_missing_both]
    data.loc[mask_missing_both, 'host_listings_count'] = calc_total[mask_missing_both]
    
    # If still missing, set to 1 (assuming single listing)
    data['host_total_listings_count'] = data['host_total_listings_count'].fillna(1)
    data['host_listings_count'] = data['host_listings_count'].fillna(1)
    
    # Calculate summary of changes
    final_missing = {
        'host_is_superhost': data['host_is_superhost'].isna().sum(),
        'has_availability': data['has_availability'].isna().sum(),
        'host_total_listings_count': data['host_total_listings_count'].isna().sum(),
        'host_listings_count': data['host_listings_count'].isna().sum()
    }
    
    summary = {
        'original_missing': original_missing,
        'final_missing': final_missing,
        'changes': {
            col: original_missing[col] - final_missing[col] 
            for col in original_missing.keys()
        }
    }
    
    return data, summary

# Read your data
df = pd.read_csv('cleaned_data/prepared/acc_bath_processed_listings_data.csv')

# Apply the cleaning
df_cleaned, summary = clean_host_and_availability_data(df)

# Print summary of changes
print("\nCleaning Summary:")
for col in summary['changes']:
    print(f"\n{col}:")
    print(f"  Original missing: {summary['original_missing'][col]}")
    print(f"  Values fixed: {summary['changes'][col]}")
    print(f"  Remaining missing: {summary['final_missing'][col]}")

# Save the cleaned data
df_cleaned.to_csv('cleaned_data/prepared/final_processed_listings_data.csv', index=False)


Cleaning Summary:

host_is_superhost:
  Original missing: 398
  Values fixed: 398
  Remaining missing: 0

has_availability:
  Original missing: 6380
  Values fixed: 6380
  Remaining missing: 0

host_total_listings_count:
  Original missing: 10
  Values fixed: 10
  Remaining missing: 0

host_listings_count:
  Original missing: 10
  Values fixed: 10
  Remaining missing: 0


# Resulting missing values

In [41]:
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV data
df = pd.read_csv('cleaned_data/prepared/final_processed_listings_data.csv')
# Calculate the percentage of missing values for each column
missing_percentages = df.isnull().mean() * 100

# Sort the columns by percentage of missing values (descending order)
missing_percentages_sorted = missing_percentages.sort_values(ascending=False)

# Filter to include only columns with missing values BELOW 35%
columns_with_missing = missing_percentages[(missing_percentages > 0) & (missing_percentages <= 35)].index
df_missing = df[columns_with_missing]

# Create a heatmap of missing values with a clearer color scheme
plt.figure(figsize=(12, 8))
sns.heatmap(df_missing.isnull(), cbar=False, yticklabels=False, cmap='Reds')
plt.title('Missing Values Heatmap (Columns with 1-35% Missing Values)')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.savefig('missing_values_heatmap.png')
plt.close()

# Create a bar plot of missing value percentages
missing_percentages_filtered = missing_percentages_sorted[(missing_percentages_sorted > 0) & (missing_percentages_sorted <= 35)]
plt.figure(figsize=(12, 8))
missing_percentages_filtered.plot(kind='bar')
plt.title('Percentage of Missing Values by Column (1-35%)')
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing Values')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('missing_values_percentage.png')
plt.close()

# Print summary of missing values
print("Columns with missing values (1-35%):")
print(missing_percentages[(missing_percentages > 0) & (missing_percentages <= 35)].sort_values(ascending=False))

print("\nTotal number of missing values in filtered columns:", df_missing.isnull().sum().sum())
print("Total number of cells in filtered dataset:", df_missing.size)
print("Percentage of missing values in filtered dataset:", (df_missing.isnull().sum().sum() / df_missing.size) * 100)

# Identify columns with more than 35% missing values
columns_to_drop = missing_percentages[missing_percentages > 35].index.tolist()
print("\nColumns with more than 35% missing values (excluded from visualization):")
print(columns_to_drop)

# Save the list of columns to drop
with open('columns_to_drop.txt', 'w') as f:
    for column in columns_to_drop:
        f.write(f"{column}\n")

print("\nAnalysis complete. Check the generated images and 'columns_to_drop.txt' file.")

Columns with missing values (1-35%):
bathrooms    28.440120
price        19.671897
dtype: float64

Total number of missing values in filtered columns: 57159
Total number of cells in filtered dataset: 237608
Percentage of missing values in filtered dataset: 24.05600821521161

Columns with more than 35% missing values (excluded from visualization):
['first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']

Analysis complete. Check the generated images and 'columns_to_drop.txt' file.


In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def analyze_listing_counts(df):
    """
    Analyze differences between host_total_listings_count and host_listings_count
    
    Parameters:
    df: pandas DataFrame with Airbnb listing data
    
    Returns:
    dict: Analysis results
    """
    # Create a copy to avoid modifying original
    data = df.copy()
    
    # Convert columns to numeric if they aren't already
    data['host_total_listings_count'] = pd.to_numeric(data['host_total_listings_count'], errors='coerce')
    data['host_listings_count'] = pd.to_numeric(data['host_listings_count'], errors='coerce')
    
    # Calculate basic statistics
    total_rows = len(data)
    matching_rows = (data['host_total_listings_count'] == data['host_listings_count']).sum()
    different_rows = total_rows - matching_rows
    
    # Calculate differences where they exist
    data['listing_count_diff'] = data['host_total_listings_count'] - data['host_listings_count']
    
    # Get statistics about differences
    diff_stats = data['listing_count_diff'].describe()
    
    # Find examples of largest differences
    largest_diff = (
        data[data['listing_count_diff'] != 0]
        .sort_values('listing_count_diff', ascending=False)
        .head(10)
    )
    
    results = {
        'total_listings': total_rows,
        'matching_counts': matching_rows,
        'different_counts': different_rows,
        'matching_percentage': (matching_rows / total_rows * 100).round(2),
        'difference_stats': diff_stats,
        'largest_differences': largest_diff
    }
    
    return results, data['listing_count_diff']

def plot_listing_count_differences(diff_series):
    """
    Create visualizations of the differences between listing counts
    """
    plt.figure(figsize=(12, 6))
    
    # Create histogram of differences, excluding zeros
    non_zero_diff = diff_series[diff_series != 0]
    plt.hist(non_zero_diff, bins=50, edgecolor='black')
    plt.title('Distribution of Differences Between\nTotal Listings Count and Listings Count')
    plt.xlabel('Difference (Total - Regular)')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    return plt.gcf()

# Example usage
df = pd.read_csv('cleaned_data/prepared/final_processed_listings_data.csv')
results, diff_series = analyze_listing_counts(df)

# Print summary
print("\nListing Count Comparison Summary:")
print(f"Total listings analyzed: {results['total_listings']:,}")
print(f"Listings with matching counts: {results['matching_counts']:,} ({results['matching_percentage']}%)")
print(f"Listings with different counts: {results['different_counts']:,} ({100-results['matching_percentage']:.2f}%)")

print("\nDifference Statistics:")
print(results['difference_stats'])

print("\nTop 10 Largest Differences:")
print(results['largest_differences'][['host_id', 'host_total_listings_count', 
                                    'host_listings_count', 'listing_count_diff']])

# Create and save visualization
plt.figure(figsize=(12, 6))
plot_listing_count_differences(diff_series)
plt.savefig('listing_count_differences.png', dpi=300, bbox_inches='tight')
plt.close()


Listing Count Comparison Summary:
Total listings analyzed: 118,804
Listings with matching counts: 71,550 (60.23%)
Listings with different counts: 47,254 (39.77%)

Difference Statistics:
count    118804.000000
mean          9.498872
std          67.009717
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max        3748.000000
Name: listing_count_diff, dtype: float64

Top 10 Largest Differences:
         host_id  host_total_listings_count  host_listings_count  \
99805  439074505                     6278.0               2530.0   
84879  439074505                     6278.0               2530.0   
14186  439074505                     6278.0               2530.0   
89734  439074505                     6278.0               2530.0   
29063  439074505                     6278.0               2530.0   
29019  439074505                     6278.0               2530.0   
16089  439074505                     6278.0               2530.0   
50639  40219131

<Figure size 1200x600 with 0 Axes>

# Remove the faulty price column

In [3]:
# Read the main dataset
main_df = pd.read_csv('cleaned_data/prepared/final_processed_listings_data.csv')

# Show the current columns (optional, for verification)
print("Current columns:", main_df.columns.tolist())

# Remove existing price column
if 'price' in main_df.columns:
    main_df = main_df.drop(columns=['price'])
    print("\nRemoved existing 'price' column")

# Save back to the same file
main_df.to_csv('final_processed_listings_data.csv', index=False)
print(f"Saved dataset with {len(main_df)} rows and {len(main_df.columns)} columns")

# Verify the column was removed (optional)
print("\nRemaining columns:", main_df.columns.tolist())

Current columns: ['id', 'last_scraped', 'name', 'host_id', 'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_ho

In [6]:
# Read the main dataset
main_df = pd.read_csv('cleaned_data/prepared/final_processed_listings_data.csv')

# Show the current columns (optional, for verification)
print("Current columns:", main_df.columns.tolist())

main_df = main_df.drop(columns=['beds_imputed_method', 'bedrooms_imputed_method', 'calendar_last_scraped'])

# Save back to the same file
main_df.to_csv('final_processed_listings_data.csv', index=False)
print(f"Saved dataset with {len(main_df)} rows and {len(main_df.columns)} columns")

# Verify the column was removed (optional)
print("\nRemaining columns:", main_df.columns.tolist())

Current columns: ['Unnamed: 0', 'id', 'last_scraped', 'name', 'host_id', 'host_is_superhost', 'host_listings_count', 'host_total_listings_count', 'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 